# Credit Card Fraud Prediction - Loading Dataset using Snowpark Python

This example is based on the Machine Learning for Credit Card Fraud detection - Practical handbook, https://fraud-detection-handbook.github.io/fraud-detection-handbook/

## Loading Credit Card Transactions into Snowflake

### Import the dependencies and connect to Snowflake

In [ ]:
# Snowpark
import snowflake.snowpark as snp
from snowflake.snowpark import Session
import snowflake.snowpark.types as T
import snowflake.snowpark.functions as F

print(f"Using Snowpark for Pythjon version: {snp.__version__}")

In [ ]:
# Other
import getpass

Get connection parameters

In [ ]:
snf_accountname = input('Enter snowflake account name:') # ORGNAME-ACCOUNTNAME (separated by minus sign)

In [ ]:
snf_user = input('Enter Snowflake user:')

In [ ]:
snf_password = getpass.getpass()

In [ ]:
snf_role = input('Enter Snowflake role:')

In [ ]:
snf_db = input('Enter Database:')

In [ ]:
snf_schema = input('Enter schema:')

In [ ]:
snf_wh = input('Enter Warehouse:')

In [ ]:
connection_parameters = {
    "account": snf_accountname,
    "user": snf_user,
    "password": snf_password,
    "role": snf_role,
    "database": snf_db,
    "schema": snf_schema,
    "warehouse": snf_wh
}

session = Session.builder.configs(connection_parameters).create()

The **get_** functions can be use to get information about the current database, schema, role etc

In [ ]:
print(f"Current schema: {session.get_fully_qualified_current_schema()}")
print(f"current role: {session.get_current_role()}")
print(f"current warehouse: {session.get_current_warehouse()}")

### Define Staging Area and the Schema for the transaction table

Using SQL we can create a internal stage and then use the **put** function to uplad the **fraud_transactions.csv.gz** file to it.

In [ ]:
stage_name = "FRAUD_DATA"
# Create a internal staging area for uploading the source file
session.sql(f"CREATE or replace STAGE {stage_name}").collect()

# Upload the source file to the stage
putResult = session.file.put("data/fraud_transactions.csv.gz", f"@{stage_name}", auto_compress=False, overwrite=True)

putResult

Define the schma for our **CUSTOMER_TRANSACTIONS_FRAUD** table

In [ ]:
# Define the schema for the Frauds table
dfCustTrxFraudSchema = T.StructType(
    [
        T.StructField("TRANSACTION_ID", T.IntegerType()),
        T.StructField("TX_DATETIME", T.TimestampType()),
        T.StructField("CUSTOMER_ID", T.IntegerType()),
        T.StructField("TERMINAL_ID", T.IntegerType()),
        T.StructField("TX_AMOUNT", T.FloatType()),
        T.StructField("TX_TIME_SECONDS", T.IntegerType()),
        T.StructField("TX_TIME_DAYS", T.IntegerType()),
        T.StructField("TX_FRAUD", T.IntegerType()),
        T.StructField("TX_FRAUD_SCENARIO", T.IntegerType())
    ]
)

Load the **fraud_transactions.csv.gz** to a DataFrame reader and save into a table

In [ ]:
# Crete a reader
dfReader = session.read.schema(dfCustTrxFraudSchema)

# Get the data into the data frame
dfCustTrxFraudRd = dfReader.csv(f"@{stage_name}/fraud_transactions.csv.gz")

In [ ]:
# Write the dataframe in a table
ret = dfCustTrxFraudRd.write.mode("overwrite").saveAsTable("CUSTOMER_TRANSACTIONS_FRAUD")

### Read the data from the staging area and create CUSTOMER_TRANSACTIONS_FRAUD, CUSTOMERS and TERMINALS tables

In [ ]:
# Now create Customers and Terminal tables

dfCustTrxFraudTb =session.table("CUSTOMER_TRANSACTIONS_FRAUD")

dfCustomers = dfCustTrxFraudTb.select(F.col("CUSTOMER_ID")).distinct().sort(F.col("CUSTOMER_ID"))

dfTerminals = dfCustTrxFraudTb.select(F.col("TERMINAL_ID")).distinct().sort(F.col("TERMINAL_ID"))
                                
ret2 = dfCustomers.write.mode("overwrite").saveAsTable("CUSTOMERS")

ret3 = dfTerminals.write.mode("overwrite").saveAsTable("TERMINALS")

In [ ]:
dfCustTrxFraudTb.show()

In [ ]:
session.close()